<a href="https://colab.research.google.com/github/jmcaussade/Xinergy_Clasificacion_glosas/blob/main/ZeroShotV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers


In [10]:
import pandas as pd
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

#Carga de archivo Google Colab

#Montar Drive

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Hugging Face authentication

In [4]:
from google.colab import userdata
from huggingface_hub import login

In [5]:
# Retrieve the Hugging Face token
hf_token = userdata.get('HuggingFace')  # Replace 'HF_TOKEN' with your actual secret name
login(hf_token)

# Carga de archivo

In [6]:
ruta_data_set = '/content/drive/MyDrive/Xinergy/Proyecto_Clasificacion_Glosa/Industria_Financiera/Recoleccion_Datos_copy.xlsx'
df = pd.read_excel(ruta_data_set, sheet_name='M0_Tecnologia')
df.head()
df

,Glosa,Categoria M1,Categoria M2,Obs,Revision Cote
0,SERVICIO MONITOREO PREVENCION FRAUDES RAPPI,Servicios,Ciberseguridad,NaN,ok
1,CONSULTORIAS SEGURIDAD (EXENTO),Servicios,Ciberseguridad,NaN,ok
2,SERVICIO IMPLEMENTACIÓN SEGURIDAD IPS,Servicios,Ciberseguridad,NaN,ok
3,SERV EXTER SUBCONTRATADO SEGURIDAD TECNOLO...,Servicios,Ciberseguridad,NaN,ok
4,ANEXO 12 PHISHING TASK FORCE,Servicios,Ciberseguridad,NaN,ok
5,Gastos Telefonia Fija,Telecom,Enlace,NaN,ok
6,SERVICIOS DE TELEFONIA - ENLACES TELEFONICOS,Telecom,Enlace,NaN,ok
7,SERVICIOS DE TELEFONIA - LINEAS ANALOGOS Y...,Telecom,Enlace,NaN,ok
8,SERVICIOS DE TELEFONIA - LINEAS PRIVADAS,Telecom,Enlace,NaN,ok
9,00160 SERVICIO DE TELEFONÍA FIJA LARGA DISTANC...,Telecom,Enlace,NaN,ok


# Definicion Categorias

In [7]:
# Define classes (categories)
labels = ["Ciberseguridad", "Enlace", "Impresion", "Licencias", "Microinformatica", "Movil", "Nube", "Servidores", "Soporte", "Atm"]

# Models

In [20]:
# Load zero-shot classification models
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available
models = {
    # Hugging Face pipelines
    "facebook_bart_large_mnli": pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device),
    "joeddav_xlm_roberta_large_xnli": pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device=device),
    "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli": pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=device),
    "cross-encoder/nli-deberta-v3-small": pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-small", device=device),

    # Sentence Transformers for semantic similarity
    "sentence_transformer_all_mpnet_base_v2": SentenceTransformer('all-mpnet-base-v2'),
    "sentence_transformer_multi_qa_mpnet_base_dot_v1": SentenceTransformer('multi-qa-mpnet-base-dot-v1')
}

Device set to use cuda:0
Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Results dataframe

In [21]:
# Create the results DataFrame
sentences = df['Glosa'].tolist()
correct_categories = df['Categoria M2'].tolist()  # Extract the correct categories from the dataframe

results_df = pd.DataFrame({
    "sentences": sentences,
    "correct_category": correct_categories  # Add the correct categories column
})


In [24]:
# Classify sentences in batches
for model_name, model in models.items():
    if "sentence_transformer" in model_name:
        # For SentenceTransformer models, use cosine similarity
        best_labels = []
        sentence_embeddings = model.encode(results_df['sentences'].tolist(), convert_to_tensor=True)
        label_embeddings = model.encode(labels, convert_to_tensor=True)
        for sentence_embedding in sentence_embeddings:
            similarity_scores = util.pytorch_cos_sim(sentence_embedding, label_embeddings)
            best_label_idx = similarity_scores.argmax().item()
            best_labels.append(labels[best_label_idx])
    else:
        # For Hugging Face pipeline models
        results = model(results_df['sentences'].tolist(), candidate_labels=labels)
        best_labels = [result['labels'][0] for result in results]

    # Add results as a new column in the DataFrame
    results_df[model_name] = best_labels

In [25]:
results_df

,sentences,correct_category,facebook_bart_large_mnli,joeddav_xlm_roberta_large_xnli,MoritzLaurer/mDeBERTa-v3-base-mnli-xnli,cross-encoder/nli-deberta-v3-small,sentence_transformer_all_mpnet_base_v2,sentence_transformer_multi_qa_mpnet_base_dot_v1
0,SERVICIO MONITOREO PREVENCION FRAUDES RAPPI,Ciberseguridad,Servidores,Servidores,Servidores,Impresion,Servidores,Licencias
1,CONSULTORIAS SEGURIDAD (EXENTO),Ciberseguridad,Servidores,Soporte,Soporte,Servidores,Licencias,Licencias
2,SERVICIO IMPLEMENTACIÓN SEGURIDAD IPS,Ciberseguridad,Servidores,Servidores,Servidores,Servidores,Servidores,Servidores
3,SERV EXTER SUBCONTRATADO SEGURIDAD TECNOLO...,Ciberseguridad,Servidores,Servidores,Servidores,Servidores,Servidores,Servidores
4,ANEXO 12 PHISHING TASK FORCE,Ciberseguridad,Atm,Soporte,Enlace,Servidores,Nube,Impresion
5,Gastos Telefonia Fija,Enlace,Atm,Movil,Movil,Servidores,Ciberseguridad,Servidores
6,SERVICIOS DE TELEFONIA - ENLACES TELEFONICOS,Enlace,Enlace,Enlace,Servidores,Licencias,Servidores,Servidores
7,SERVICIOS DE TELEFONIA - LINEAS ANALOGOS Y...,Enlace,Servidores,Servidores,Servidores,Enlace,Servidores,Servidores
8,SERVICIOS DE TELEFONIA - LINEAS PRIVADAS,Enlace,Servidores,Servidores,Servidores,Enlace,Servidores,Servidores
9,00160 SERVICIO DE TELEFONÍA FIJA LARGA DISTANC...,Enlace,Servidores,Servidores,Servidores,Licencias,Servidores,Licencias


In [27]:
# Calculate performance for each model
performance = {}

for model_name in models.keys():
    # Compare model predictions with correct_category
    correct_predictions = (results_df[model_name] == results_df["correct_category"]).sum()
    total_predictions = len(results_df)
    accuracy = (correct_predictions / total_predictions) * 100  # Calculate accuracy percentage

    # Store results in the performance dictionary
    performance[model_name] = {
        "correct_answers": correct_predictions,
        "total_predictions": total_predictions,
        "accuracy_percent": accuracy
    }

# Display the results
performance_df = pd.DataFrame.from_dict(performance, orient="index")
performance_df.reset_index(inplace=True)
performance_df.rename(columns={"index": "model_name"}, inplace=True)

In [28]:
performance_df

,model_name,correct_answers,total_predictions,accuracy_percent
0,facebook_bart_large_mnli,19,53,35.849057
1,joeddav_xlm_roberta_large_xnli,19,53,35.849057
2,MoritzLaurer/mDeBERTa-v3-base-mnli-xnli,16,53,30.188679
3,cross-encoder/nli-deberta-v3-small,12,53,22.641509
4,sentence_transformer_all_mpnet_base_v2,16,53,30.188679
5,sentence_transformer_multi_qa_mpnet_base_dot_v1,15,53,28.301887
